In [ ]:
import pandas as pd
#1. Read_csv
data= pd.read_csv("df_temp.csv")

#2. Train, test split
train_mask = (data['기준년월'] != 202006)
X_train = data.loc[train_mask, :]
X_test = data.loc[~train_mask, :]
#X_train = X_train.drop('기준년월', axis=1)
#X_test = X_test.drop('기준년월', axis=1)

print("학습사이즈 : ", len(X_train))
print("테스트사이즈 : ", len(X_test))

#3. Y split
X_train_x = X_train.drop(['매수고객수', '월'], axis = 1)
X_train_y = X_train['매수고객수']
X_test_x = X_test.drop(['매수고객수', '월'], axis = 1)
X_test_y = X_test['매수고객수']


X_train_x

In [ ]:
X_train_x.columns

In [ ]:
categ = [5, 6, 7, 8, 9,10,11]

from catboost import CatBoostClassifier, CatBoostRegressor

cb_model = CatBoostRegressor(iterations=1300,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 23,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             od_wait=100)

cb_model.fit(X_train_x, X_train_y,
             eval_set=(X_test_x,X_test_y),
             cat_features=categ,
             use_best_model=True,
             verbose=True)

In [ ]:
temp = pd.read_csv("template.csv")
#temp = temp.drop(['그룹내고객수'], axis = 1)

y_test = cb_model.predict(temp)
a = pd.DataFrame(y_test, columns=['매수고객수'])
result = pd.concat([temp, a], axis = 1)

ou = result[['그룹번호', '종목번호', '매수고객수']].sort_values(by=['그룹번호','매수고객수'], 
                                                    ascending = [True, False])

result = ou.groupby(['그룹번호']).head(3)
# result
ou

In [ ]:
result

In [ ]:
stock_name = pd.read_csv("종목변환.csv")
group_name = pd.read_csv("그룹변환.csv")

#result
output = pd.DataFrame(columns=['그룹번호','Top1','Top2','Top3'])

top_list = []
top = []
count = 0
for idx, col in result.iterrows():
    if count % 3 == 0:
        count +=1
        top.append(group_name.iloc[col['그룹번호'].astype(int), 1])
        top.append(stock_name.iloc[col['종목번호'].astype(int), 1])
    elif count % 3 == 1:
        count +=1
        top.append(stock_name.iloc[col['종목번호'].astype(int), 1])
    else:
        top.append(stock_name.iloc[col['종목번호'].astype(int), 1])
        top_list.append(top)
        #clear
        top = []
        count = 0
        
for idx, top in enumerate(top_list):
    output.loc[idx] = top

output.to_csv("classif_train_result.csv", mode='w', header=True, index=False)
output